In [1]:
!pip install transformers; pip install torch; pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118;

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
import os
from PIL import Image

# Path to the directory containing the images
img_directory = "img/"

# Get the list of image files in the directory
image_files = [f for f in os.listdir(img_directory) if f.endswith(".png")]

# Choose a sample image file
sample_image_file = os.path.join(img_directory, image_files[0])

# Load the sample image
sample_image = Image.open(sample_image_file)

# Get the dimensions of the sample image
image_width, image_height = sample_image.size

# Display the dimensions of the sample image
print("Image Dimensions (Width x Height):", image_width, "x", image_height)


Image Dimensions (Width x Height): 480 x 800


In [3]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
import numpy as np

# Load the CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Helper function to preprocess image
def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.convert("RGB")
    image = image.resize((480, 800))  # Resize image to match CLIP's input size
    image = np.array(image)
    image = torch.tensor(image).unsqueeze(0)  # Add batch dimension
    return image

# Helper function to preprocess text
def preprocess_text(text):
    inputs = clip_processor(text, return_tensors="pt", padding=True, truncation=True)
    return inputs


2024-04-23 22:48:39.769421: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 22:48:39.769485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 22:48:39.770545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 22:48:39.777372: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 22:48:40.931094: W tensorflow/compiler/tf2

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [4]:
import json

# Load training data from train.jsonl
def load_training_data(file_path):
    with open(file_path, 'r') as f:
        data = [json.loads(line) for line in f]
    return data

train_data = load_training_data('train.jsonl')

# Print the first few examples to verify the data loading
print("Number of training examples:", len(train_data))
print("Example:", train_data[0])


Number of training examples: 8500
Example: {'id': '42953', 'img': 'img/42953.png', 'label': 0, 'text': 'its their character not their color that matters'}


In [5]:
from torch.utils.data import DataLoader
from tqdm import tqdm

# Preprocess the training data
def preprocess_training_data(data):
    preprocessed_images = []
    preprocessed_texts = []
    labels = []
    
    for example in tqdm(data, desc="Preprocessing training data"):
        image_path = example['img']
        text = example['text']
        label = example['label']
        
        # Preprocess image
        image = preprocess_image(image_path)
        preprocessed_images.append(image)
        
        # Preprocess text
        text_inputs = preprocess_text(text)
        preprocessed_texts.append(text_inputs)
        
        labels.append(label)
    
    return preprocessed_images, preprocessed_texts, labels

preprocessed_images, preprocessed_texts, labels = preprocess_training_data(train_data)


Preprocessing training data:   0%|                                                             | 0/8500 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'img/42953.png'